[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/EMQA/blob/main/EMQA_case_perf/EMQA_case_perf.ipynb)

# EMQA_case_perf
Strategy performance analysis: cumulative returns, drawdown, rolling Sharpe.
**Output:** `case_perf.pdf`

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

plt.rcParams.update({
    'figure.facecolor': 'none',
    'axes.facecolor': 'none',
    'savefig.facecolor': 'none',
    'savefig.transparent': True,
    'axes.grid': False,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'font.size': 11,
    'figure.figsize': (12, 6),
})

COLORS = {
    'blue': '#1A3A6E', 'red': '#CD0000', 'green': '#2E7D32',
    'orange': '#E67E22', 'purple': '#8E44AD', 'gray': '#808080',
    'cyan': '#00BCD4', 'amber': '#B5853F'
}

def save_fig(fig, name):
    fig.savefig(name, bbox_inches='tight', transparent=True, dpi=300)
    print(f"Saved: {name}")


In [ ]:
import yfinance as yf

def fetch(ticker, start='2020-01-01', end='2025-12-31'):
    d = yf.download(ticker, start=start, end=end, progress=False)
    if isinstance(d.columns, pd.MultiIndex):
        return d['Close'].squeeze().dropna()
    return d['Close'].dropna()


In [ ]:
# Fetch Brent and compute momentum strategy (same as case_momentum)
brent = fetch('BZ=F', start='2018-01-01')
log_ret = np.log(brent / brent.shift(1)).dropna()

mom_20 = log_ret.rolling(20).sum()
signal = (mom_20 > 0).astype(float).shift(1).fillna(0)
strat_ret = signal * log_ret
buyhold_ret = log_ret

# Cumulative wealth
cum_strat = np.exp(strat_ret.cumsum())
cum_buyhold = np.exp(buyhold_ret.cumsum())

# Drawdown
peak_strat = cum_strat.cummax()
dd_strat = (cum_strat - peak_strat) / peak_strat * 100

peak_bh = cum_buyhold.cummax()
dd_bh = (cum_buyhold - peak_bh) / peak_bh * 100

# Rolling 60-day Sharpe
roll_sharpe_strat = strat_ret.rolling(60).mean() / strat_ret.rolling(60).std() * np.sqrt(252)
roll_sharpe_bh = buyhold_ret.rolling(60).mean() / buyhold_ret.rolling(60).std() * np.sqrt(252)

# Summary stats
total_ret_strat = (cum_strat.iloc[-1] - 1) * 100
total_ret_bh = (cum_buyhold.iloc[-1] - 1) * 100
max_dd_strat = dd_strat.min()
max_dd_bh = dd_bh.min()
sharpe_strat = strat_ret.mean() / strat_ret.std() * np.sqrt(252)
sharpe_bh = buyhold_ret.mean() / buyhold_ret.std() * np.sqrt(252)

print(f"{'Metric':<25} {'Momentum':>12} {'Buy & Hold':>12}")
print("-" * 50)
print(f"{'Total Return (%)':<25} {total_ret_strat:>12.1f} {total_ret_bh:>12.1f}")
print(f"{'Max Drawdown (%)':<25} {max_dd_strat:>12.1f} {max_dd_bh:>12.1f}")
print(f"{'Sharpe Ratio':<25} {sharpe_strat:>12.3f} {sharpe_bh:>12.3f}")

In [ ]:
# 3-panel vertical
fig, axes = plt.subplots(3, 1, figsize=(12, 12), sharex=True)

# Panel 1 - Cumulative returns
axes[0].plot(cum_strat.index, (cum_strat - 1) * 100, color=COLORS['blue'], linewidth=1.5,
             label=f'Momentum ({total_ret_strat:.0f}%)')
axes[0].plot(cum_buyhold.index, (cum_buyhold - 1) * 100, color=COLORS['gray'], linewidth=1.5,
             label=f'Buy & Hold ({total_ret_bh:.0f}%)')
axes[0].axhline(0, color=COLORS['gray'], linestyle=':', linewidth=0.8)
axes[0].set_ylabel('Cumulative Return (%)')
axes[0].set_title('Strategy Performance Analysis - Brent Crude Momentum')
axes[0].legend(loc='upper center', bbox_to_anchor=(0.5, -0.05), frameon=False, ncol=2)

# Panel 2 - Drawdown
axes[1].fill_between(dd_strat.index, dd_strat.values, 0, color=COLORS['blue'], alpha=0.4,
                      label=f'Momentum (max={max_dd_strat:.1f}%)')
axes[1].fill_between(dd_bh.index, dd_bh.values, 0, color=COLORS['gray'], alpha=0.3,
                      label=f'Buy & Hold (max={max_dd_bh:.1f}%)')
axes[1].set_ylabel('Drawdown (%)')
axes[1].set_title('Drawdown')
axes[1].legend(loc='upper center', bbox_to_anchor=(0.5, -0.05), frameon=False, ncol=2)

# Panel 3 - Rolling 60-day Sharpe
axes[2].plot(roll_sharpe_strat.index, roll_sharpe_strat.values, color=COLORS['blue'],
             linewidth=1, label='Momentum')
axes[2].plot(roll_sharpe_bh.index, roll_sharpe_bh.values, color=COLORS['gray'],
             linewidth=1, label='Buy & Hold')
axes[2].axhline(0, color=COLORS['gray'], linestyle=':', linewidth=0.8)
axes[2].set_ylabel('Sharpe Ratio')
axes[2].set_xlabel('Date')
axes[2].set_title('Rolling 60-day Sharpe Ratio')
axes[2].legend(loc='upper center', bbox_to_anchor=(0.5, -0.10), frameon=False, ncol=2)

plt.tight_layout()
save_fig(fig, 'case_perf.pdf')
plt.show()